In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

import lightgbm as lgb
from lightgbm import LGBMRegressor

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import BallTree
from sklearn.metrics import (
    mean_squared_error,
    roc_auc_score,
    r2_score,
)

import holidays

DATA_DIR = Path('../shared_data/')  # change to your local path
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [ ]:
# ---- 1.1 Load wildfires ----
fires_path = DATA_DIR / 'wildfires_fires.parquet'

fires = pd.read_parquet(
    fires_path,
    columns=[
        'FIRE_SIZE',
        'DISCOVERY_DATE',
        'STATE',
        'STAT_CAUSE_CODE',
        'LATITUDE',
        'LONGITUDE',
    ],
)

# Convert DISCOVERY_DATE to datetime
if not np.issubdtype(fires['DISCOVERY_DATE'].dtype, np.datetime64):
    fires['DISCOVERY_DATE'] = pd.to_datetime(fires['DISCOVERY_DATE'])

fires = fires.rename(columns={'DISCOVERY_DATE': 'date'})

# Filter date range 1992–2015
fires = fires[
    (fires['date'].dt.year >= 1992) & (fires['date'].dt.year <= 2015)
]

# Drop rows without coordinates
fires = fires.dropna(subset=['LATITUDE', 'LONGITUDE'])
fires.head()

In [ ]:
# ---- 1.1b Attach closest weather station to each fire ----
cities_path = DATA_DIR / 'cities.csv'
stations = pd.read_csv(cities_path)

# Keep one row per station with coordinates
stations = stations[["station_id", "latitude", "longitude"]].drop_duplicates('station_id')

# Build BallTree (coordinates in radians)
station_coords = np.radians(stations[['latitude', 'longitude']].values)
fire_coords = np.radians(fires[['LATITUDE', 'LONGITUDE']].values)

tree = BallTree(station_coords, metric='haversine')
dist_rad, ind = tree.query(fire_coords, k=1)

fires['station_id'] = stations.iloc[ind.flatten()]['station_id'].values
fires['dist_to_station_km'] = dist_rad.flatten() * 6371.0  # earth radius ~6371 km

fires.head()

In [ ]:
# ---- 1.2 Aggregate to station–day ----

# Basic daily targets per station
daily_agg = (
    fires.groupby(['station_id', 'date'])
    .agg(
        n_fires=('FIRE_SIZE', 'size'),
        area_burned=('FIRE_SIZE', 'sum'),
    )
    .reset_index()
)

# Cause counts per station–day (wide format)
cause_counts = (
    fires.pivot_table(
        index=['station_id', 'date'],
        columns='STAT_CAUSE_CODE',
        values='FIRE_SIZE',  # any column, we just count
        aggfunc='count',
        fill_value=0,
    )
    .rename_axis(columns='cause_code')
)

# Make nicer column names: cause_1, cause_2, ...
cause_counts.columns = [f'cause_{int(c)}' for c in cause_counts.columns]
cause_counts = cause_counts.reset_index()

# Merge counts into daily_agg
fires_daily = daily_agg.merge(
    cause_counts, on=['station_id', 'date'], how='left'
).fillna(0)

fires_daily['any_fire'] = (fires_daily['n_fires'] > 0).astype(int)
fires_daily.head()

In [ ]:
# ---- 2.1 Load weather (station-based) ----
weather_path = DATA_DIR / 'us_daily_weather_1992_2015.parquet'
weather = pd.read_parquet(weather_path)

# Ensure date is datetime
weather['date'] = pd.to_datetime(weather['date'])

# Restrict to 1992–2015
weather = weather[
    (weather['date'].dt.year >= 1992) & (weather['date'].dt.year <= 2015)
]

# Load station metadata (cities.csv)
cities_path = DATA_DIR / 'cities.csv'
cities_df = pd.read_csv(cities_path)

# Merge state info into weather via station_id
weather = weather.merge(
    cities_df[['station_id', 'state']].drop_duplicates('station_id'),
    on='station_id',
    how='left',
)

weather = weather.rename(
    columns={
        'state': 'STATE',
        'avg_temp_c': 'tavg',
        'min_temp_c': 'tmin',
        'max_temp_c': 'tmax',
        'precipitation_mm': 'prcp_1d',  # explicit 1-day precipitation
        'avg_wind_speed_kmh': 'wspd',
        'avg_sea_level_pres_hpa': 'pres',
    }
)

weather.head()

In [ ]:
# ---- 2.2 Aggregate station data to station–day ----
weather_daily = (
    weather.groupby(['station_id', 'date'])
    .agg(
        tavg=('tavg', 'mean'),
        tmin=('tmin', 'mean'),
        tmax=('tmax', 'mean'),
        prcp_1d=('prcp_1d', 'sum'),  # daily precip (1-day)
        wspd=('wspd', 'mean'),
        pres=('pres', 'mean'),
        STATE=('STATE', 'first'),
    )
    .reset_index()
)

weather_daily.head()

In [ ]:
# ---- 2.3 Add lagged weather features (per station) ----
def add_lagged_features(
    df,
    group_col,
    date_col,
    base_cols,
    windows=(3, 7, 30),
    shift=1,
):
    """
    For each base_col, compute rolling means or sums over given windows,
    then shift by `shift` days to avoid using same-day info.
    """
    df = df.sort_values([group_col, date_col]).copy()

    for col in base_cols:
        for w in windows:
            roll = (
                df.groupby(group_col)[col]
                .transform(lambda x: x.rolling(w, min_periods=1).mean())
            )
            df[f'{col}_mean_{w}'] = roll.shift(shift)

    # Cumulative precipitation over 7 and 30 days, based on 1-day precip
    for w in [7, 30]:
        roll = (
            df.groupby(group_col)['prcp_1d']
            .transform(lambda x: x.rolling(w, min_periods=1).sum())
        )
        df[f'prcp_1d_sum_{w}'] = roll.shift(shift)

    return df

weather_feat = weather_daily.copy()

# Impute any remaining NaNs before building lags (per station)
weather_feat = weather_feat.set_index('station_id')
weather_feat = weather_feat.groupby(level=0).ffill().bfill()
weather_feat = weather_feat.reset_index()

weather_feat['year'] = weather_feat['date'].dt.year
weather_feat['month'] = weather_feat['date'].dt.month
weather_feat['doy'] = weather_feat['date'].dt.dayofyear

base_cols = ['tavg', 'tmin', 'tmax', 'prcp_1d', 'wspd', 'pres']

weather_feat = add_lagged_features(
    weather_feat,
    group_col='station_id',
    date_col='date',
    base_cols=base_cols,
)

# Drop first few rows where lags are NaN (from shift)
weather_feat = weather_feat.dropna().reset_index(drop=True)

weather_feat.head()

In [ ]:
# ---- 3.1 Merge weather and fires on station + date + calendar / holiday features ----
data = weather_feat.merge(
    fires_daily, on=['station_id', 'date'], how='left'
)

# Replace NaNs in fire-related columns with 0
fire_cols = [c for c in data.columns if c.startswith('cause_')] + [
    'n_fires', 'area_burned', 'any_fire',
]
for c in fire_cols:
    if c in data.columns:
        data[c] = data[c].fillna(0)

# Transformed target for area
data['log_area_burned'] = np.log1p(data['area_burned'])

# Weekend indicator
data['is_weekend'] = data['date'].dt.weekday.isin([5, 6]).astype(int)

# US holidays
us_holidays = holidays.US()
data['holiday_name'] = data['date'].dt.date.map(us_holidays.get)
data['is_holiday'] = data['holiday_name'].notna().astype(int)
data['is_july4'] = (data['holiday_name'] == 'Independence Day').astype(int)

# Combined "weekend or holiday" flag (optional extra feature)
data['is_weekend_or_holiday'] = (
    (data['is_weekend'] == 1) | (data['is_holiday'] == 1)
).astype(int)

data.head()

In [ ]:
# ---- 4.1 Encode STATE and station_id ----
le_state = LabelEncoder()
data['STATE_LE'] = le_state.fit_transform(
    data['STATE'].fillna('UNK').astype(str)
 )

le_station = LabelEncoder()
data['station_LE'] = le_station.fit_transform(
    data['station_id'].astype(str)
 )

# ---- 4.2 Final feature list (excluding targets) ----
exclude_cols = {
    'n_fires',
    'any_fire',
    'area_burned',
    'log_area_burned',
    'STATE',
    'station_id',
    'date',
    'holiday_name',  # drop string column so all features are numeric
} | set([c for c in data.columns if c.startswith('cause_')])

feature_cols = [c for c in data.columns if c not in exclude_cols]
len(feature_cols), feature_cols[:10]

In [ ]:
# ---- 5.1 Train / test split by year ----
train_year_cutoff = 2011
train_mask = data['year'] < train_year_cutoff
test_mask = ~train_mask

X_train = data.loc[train_mask, feature_cols]
X_test = data.loc[test_mask, feature_cols]

y_train_count = data.loc[train_mask, 'n_fires']
y_test_count = data.loc[test_mask, 'n_fires']

y_train_bin = data.loc[train_mask, 'any_fire']
y_test_bin = data.loc[test_mask, 'any_fire']

y_train_log_area = data.loc[train_mask, 'log_area_burned']
y_test_log_area = data.loc[test_mask, 'log_area_burned']

In [ ]:
# ---- 6.1 Count model (Poisson) ----
lgb_train = lgb.Dataset(X_train, label=y_train_count)
lgb_valid = lgb.Dataset(X_test, label=y_test_count)

params_count = {
    'objective': 'poisson',
    'learning_rate': 0.05,
    'num_leaves': 64,
    'metric': 'rmse',
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'seed': RANDOM_STATE,
}

model_count = lgb.train(
    params_count,
    lgb_train,
    num_boost_round=2000,
    valid_sets=[lgb_valid],
    callbacks=[lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(period=100)],
)

pred_count = model_count.predict(X_test, num_iteration=model_count.best_iteration)
rmse_count = np.sqrt(mean_squared_error(y_test_count, pred_count))

print(f'Count model RMSE: {rmse_count:.3f}')
print(f'Mean actual count in test: {y_test_count.mean():.3f}')

In [ ]:
# ---- 6.2 Binary any-fire model (for AUC) ----
params_bin = {
    'objective': 'binary',
    'learning_rate': 0.05,
    'num_leaves': 64,
    'metric': 'auc',
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'seed': RANDOM_STATE,
}

lgb_train_bin = lgb.Dataset(X_train, label=y_train_bin)
lgb_valid_bin = lgb.Dataset(X_test, label=y_test_bin)

model_bin = lgb.train(
    params_bin,
    lgb_train_bin,
    num_boost_round=2000,
    valid_sets=[lgb_valid_bin],
    callbacks=[lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(period=100)],
)

prob_any_fire = model_bin.predict(
    X_test, num_iteration=model_bin.best_iteration
)
auc_any_fire = roc_auc_score(y_test_bin, prob_any_fire)

print(f'AUC (any_fire): {auc_any_fire:.3f}')
print(f'Positive rate (fires on a day): {y_test_bin.mean():.3f}')

In [ ]:
# ---- 7.1 Area model ----
params_area = {
    'objective': 'regression',
    'learning_rate': 0.05,
    'num_leaves': 64,
    'metric': 'rmse',
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'seed': RANDOM_STATE,
}

lgb_train_area = lgb.Dataset(X_train, label=y_train_log_area)
lgb_valid_area = lgb.Dataset(X_test, label=y_test_log_area)

model_area = lgb.train(
    params_area,
    lgb_train_area,
    num_boost_round=2000,
    valid_sets=[lgb_valid_area],
    callbacks=[lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(period=100)],
)

pred_log_area = model_area.predict(
    X_test, num_iteration=model_area.best_iteration
)

rmse_log_area = np.sqrt(mean_squared_error(
    y_test_log_area, pred_log_area
))
r2_log_area = r2_score(y_test_log_area, pred_log_area)

print(f'Area model RMSE (log1p): {rmse_log_area:.3f}')
print(f'Area model R^2 (log1p): {r2_log_area:.3f}')

# Baseline: predict mean log-area of train
baseline_log = np.full_like(y_test_log_area, y_train_log_area.mean())
baseline_rmse_log = np.sqrt(mean_squared_error(
    y_test_log_area, baseline_log
))
print(f'Baseline RMSE (log1p mean): {baseline_rmse_log:.3f}')

In [ ]:
# ---- 8.1 Prepare cause targets ----
cause_cols = [c for c in data.columns if c.startswith('cause_')]
len(cause_cols), cause_cols

# Keep only days with at least one fire
data_cause = data[data['n_fires'] > 0].copy()

# Proportions per cause
for c in cause_cols:
    data_cause[c + '_prop'] = data_cause[c] / data_cause['n_fires']

target_prop_cols = [c + '_prop' for c in cause_cols]

# Train/test split by year (same cutoff)
train_mask_cause = data_cause['year'] < train_year_cutoff
test_mask_cause = ~train_mask_cause

X_train_cause = data_cause.loc[train_mask_cause, feature_cols]
X_test_cause = data_cause.loc[test_mask_cause, feature_cols]

Y_train_cause = data_cause.loc[train_mask_cause, target_prop_cols].values
Y_test_cause = data_cause.loc[test_mask_cause, target_prop_cols].values

In [ ]:
# ---- 8.2 Multi-output regression for cause proportions ----
base_reg = LGBMRegressor(
    objective='regression',
    learning_rate=0.05,
    num_leaves=64,
    n_estimators=800,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=RANDOM_STATE,
)

multi_reg = MultiOutputRegressor(base_reg)
multi_reg.fit(X_train_cause, Y_train_cause)

Y_pred_cause = multi_reg.predict(X_test_cause)

# Ensure non-negative and normalize to sum to 1
Y_pred_cause = np.clip(Y_pred_cause, 0, None)
row_sums = Y_pred_cause.sum(axis=1, keepdims=True)
row_sums[row_sums == 0] = 1
Y_pred_cause = Y_pred_cause / row_sums

# ---- 8.3 Metrics: MSE of proportions and cross-entropy per fire ----
mse_props = mean_squared_error(Y_test_cause.reshape(-1), Y_pred_cause.reshape(-1))
print(f'MSE on cause proportions: {mse_props:.5f}')

eps = 1e-12
ce_per_day = -np.sum(Y_test_cause * np.log(Y_pred_cause + eps), axis=1)
ce_mean = ce_per_day.mean()
print(f'Mean cross-entropy (per day, per fire): {ce_mean:.3f}')

# Baseline: always predict global average cause distribution (from train)
global_dist = Y_train_cause.sum(axis=0)
global_dist = global_dist / global_dist.sum()

baseline_pred = np.tile(global_dist, (Y_test_cause.shape[0], 1))
ce_baseline = -np.sum(Y_test_cause * np.log(baseline_pred + eps), axis=1).mean()
mse_baseline = mean_squared_error(
    Y_test_cause.reshape(-1), baseline_pred.reshape(-1)
)

print(f'Baseline CE: {ce_baseline:.3f}')
print(f'Baseline MSE: {mse_baseline:.5f}')

In [ ]:
# ---- 9.1 Predicted vs Actual fire counts ----
# Assumes y_test_count and pred_count are available from earlier cells
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(6, 6))
sns.scatterplot(x=y_test_count, y=pred_count, s=5, alpha=0.4)
max_val = max(y_test_count.max(), pred_count.max())
plt.plot([0, max_val], [0, max_val], color='red', linestyle='--', label='Ideal (y = x)')
plt.xlabel('Actual fire count')
plt.ylabel('Predicted fire count')
plt.title('Predicted vs Actual Daily Fire Counts')
plt.legend()
plt.tight_layout()
plt.show()

# ---- 9.2 Predicted vs Actual log area burned ----
# Assumes y_test_log_area and pred_log_area are available
plt.figure(figsize=(6, 6))
sns.scatterplot(x=y_test_log_area, y=pred_log_area, s=5, alpha=0.4)
min_val = min(y_test_log_area.min(), pred_log_area.min())
max_val = max(y_test_log_area.max(), pred_log_area.max())
plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--', label='Ideal (y = x)')
plt.xlabel('Actual log1p(area_burned)')
plt.ylabel('Predicted log1p(area_burned)')
plt.title('Predicted vs Actual Log Area Burned')
plt.legend()
plt.tight_layout()
plt.show()

# ---- 9.3 Feature importance for area model ----
# Use LightGBM's built-in feature importance for model_area
importances_area = model_area.feature_importance(importance_type='gain')
feature_importance_area = pd.DataFrame({
    'feature': feature_cols,
    'importance': importances_area,
})
feature_importance_area = feature_importance_area.sort_values('importance', ascending=False)
top_n = 15
top_features_area = feature_importance_area.head(top_n)

plt.figure(figsize=(8, 6))
sns.barplot(data=top_features_area, x='importance', y='feature', orient='h')
plt.title(f'Top {top_n} Features for Area Model (gain importance)')
plt.xlabel('Importance (gain)')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

# ---- 9.4 Feature importance for specific causes (cause_1_prop and cause_4_prop) ----
# Extract individual regressors from multi_reg
# Assumes target_prop_cols corresponds to order of outputs in multi_reg
cause_names = ['cause_1_prop', 'cause_4_prop']
for cause_name in cause_names:
    if cause_name not in target_prop_cols:
        print(f"Warning: {cause_name} not found in target_prop_cols; skipping.")
        continue

    idx = target_prop_cols.index(cause_name)
    reg = multi_reg.estimators_[idx]  # lightgbm.LGBMRegressor instance

    fi = reg.feature_importances_
    fi_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': fi,
    }).sort_values('importance', ascending=False)
    top_k = 10
    top_fi = fi_df.head(top_k)

    plt.figure(figsize=(8, 6))
    sns.barplot(data=top_fi, x='importance', y='feature', orient='h')
    plt.title(f'Top {top_k} Features for {cause_name}')
    plt.xlabel('Importance (split count)')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.show()

# ---- 9.5 Rename cause columns to descriptive names ----
# Build mapping from STAT_CAUSE_CODE to STAT_CAUSE_DESCR from original fires dataset
fires_for_mapping = fires[['STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR']].drop_duplicates()
fires_for_mapping = fires_for_mapping.dropna(subset=['STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR'])

code_to_descr = (
    fires_for_mapping.set_index('STAT_CAUSE_CODE')['STAT_CAUSE_DESCR'].to_dict()
 )

def clean_descr(d):
    d = d.lower().strip()
    d = d.replace('&', 'and')
    d = d.replace('/', ' ')
    d = d.replace('-', ' ')
    d = d.replace(',', ' ')
    d = '_'.join(part for part in d.split() if part)
    return d

rename_map = {}
for col in data.columns:
    if col.startswith('cause_') and col.endswith('_prop') is False:
        # raw count columns like cause_1, cause_2, ...
        try:
            code = int(col.split('_')[1])
        except (IndexError, ValueError):
            continue
        if code in code_to_descr:
            descr = clean_descr(code_to_descr[code])
            rename_map[col] = f'cause_{descr}'

# Apply renaming to data and cause_cols / target_prop_cols if desired
data = data.rename(columns=rename_map)
cause_cols = [rename_map.get(c, c) for c in cause_cols]
target_prop_cols = [c + '_prop' for c in cause_cols]

# ---- 9.6 Distributions of log area burned ----
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
sns.histplot(y_train_log_area, bins=50, kde=True)
plt.title('Train log1p(area_burned)')
plt.xlabel('log1p(area_burned)')

plt.subplot(1, 2, 2)
sns.histplot(y_test_log_area, bins=50, kde=True)
plt.title('Test log1p(area_burned)')
plt.xlabel('log1p(area_burned)')

plt.tight_layout()
plt.show()

print('Train log-area:')
print(y_train_log_area.describe())
print('\nTest log-area:')
print(y_test_log_area.describe())